# SQL Generation with Transformer API

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [10]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

##It is about Jujutsu Kaisen (An anime)

In [12]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Use proper JOINs when referencing related data
- Focus on logical correctness, not style

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE sorcerers (
  sorcerer_id INTEGER PRIMARY KEY, -- Unique ID for each sorcerer
  name VARCHAR(50), -- Name of the sorcerer
  grade VARCHAR(20), -- Grade of the sorcerer (e.g. Special Grade, Grade 1)
  cursed_technique VARCHAR(100) -- Name of their cursed technique
);

CREATE TABLE curses (
  curse_id INTEGER PRIMARY KEY, -- Unique ID for each curse
  name VARCHAR(50), -- Name of the curse
  grade VARCHAR(20), -- Grade of the curse
  location VARCHAR(100) -- Where the curse was encountered
);

CREATE TABLE battles (
  battle_id INTEGER PRIMARY KEY, -- Unique ID for each battle
  sorcerer_id INTEGER, -- ID of the sorcerer
  curse_id INTEGER, -- ID of the curse
  outcome VARCHAR(10), -- Outcome of the battle (e.g. win, loss)
  battle_date DATE, -- Date the battle occurred
  domain_expansion_used BOOLEAN, -- Whether domain expansion was used
  FOREIGN KEY (sorcerer_id) REFERENCES sorcerers(sorcerer_id),
  FOREIGN KEY (curse_id) REFERENCES curses(curse_id)
);

CREATE TABLE domains (
  domain_id INTEGER PRIMARY KEY, -- Unique ID for each domain expansion
  sorcerer_id INTEGER, -- ID of the sorcerer who used the domain
  name VARCHAR(100), -- Name of the domain
  description TEXT, -- Description of the domain's abilities
  first_used DATE, -- Date the domain was first used
  FOREIGN KEY (sorcerer_id) REFERENCES sorcerers(sorcerer_id)
);

-- battles.sorcerer_id can be joined with sorcerers.sorcerer_id
-- battles.curse_id can be joined with curses.curse_id
-- domains.sorcerer_id can be joined with sorcerers.sorcerer_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


##Prompt 1 — Domain Expansion & Victory

In [13]:
question_1 = "Which sorcerers have used domain expansion in battle and won against a Special Grade curse?"
generated_sql_1 = generate_query(question_1)

print("Prompt 1 SQL:\n")
print(generated_sql_1)


Prompt 1 SQL:


SELECT s.name
FROM sorcerers s
JOIN battles b ON s.sorcerer_id = b.sorcerer_id
JOIN curses c ON b.curse_id = c.curse_id
JOIN domains d ON s.sorcerer_id = d.sorcerer_id
WHERE b.domain_expansion_used = TRUE
  AND c.grade = 'Special Grade'
  AND b.outcome = 'win'
ORDER BY s.name NULLS LAST;


##Prompt 2 — Most Defeated Curses


In [17]:
question_2 = "List all curses that have been defeated more than once, including how many times each was defeated."
generated_sql_2 = generate_query(question_2)

print("Prompt 2 SQL:\n")
print(generated_sql_2)

Prompt 2 SQL:


SELECT c.name,
       COUNT(b.outcome) AS num_defeats
FROM curses c
JOIN battles b ON c.curse_id = b.curse_id
WHERE b.outcome = 'loss'
GROUP BY c.name
HAVING COUNT(b.outcome) > 1
ORDER BY num_defeats DESC NULLS LAST;


##Prompt 3 - Wins Without Domains Expansions

In [18]:
question_3 = "Which sorcerers have never used a domain expansion in battle but still defeated a Special Grade curse?"
generated_sql_3 = generate_query(question_3)

print("Prompt 3 SQL:\n")
print(generated_sql_3)


Prompt 3 SQL:


SELECT s.name
FROM sorcerers s
JOIN battles b ON s.sorcerer_id = b.sorcerer_id
JOIN curses c ON b.curse_id = c.curse_id
WHERE b.domain_expansion_used = FALSE
  AND c.grade = 'Special Grade'
ORDER BY s.name NULLS LAST;


In [19]:
question_4 = "Which sorcerers have created more than one unique domain expansion, and what are the names of those domains?"
generated_sql_4 = generate_query(question_4)

print("New Prompt 4 SQL:\n")
print(generated_sql_4)


New Prompt 4 SQL:


SELECT s.name,
       d.name AS domain_name
FROM sorcerers s
JOIN domains d ON s.sorcerer_id = d.sorcerer_id
GROUP BY s.name,
         d.name
HAVING COUNT(DISTINCT d.domain_id) > 1;


"""
Final Exercise Report: SQL Generation using Transformers and Custom Prompting

Overview:
This lab focused on prompting the defog/sqlcoder-7b model to generate SQL queries
from natural language questions. The schema was provided in a structured format,
and prompts were designed for both real-world and fictional scenarios.

Findings:

 What Worked Well:
- When the question was directly aligned with the schema, the model generated accurate SQL.
- Joins, filters, and aggregations (GROUP BY + HAVING) were handled well.
- Jujutsu Kaisen-inspired prompts were successful when they used schema-aligned entities like
  sorcerers, curses, battles, and domain expansions.

 What Didn't Work:
- Hallucination occurred when the question relied on lore not represented in the schema.
  Example: The model incorrectly identified "Hollow Technique: Purple" as a domain expansion.
- Prompts assuming multiple domain expansions per sorcerer failed, as the schema wasn't built for that.
- When schema logic was stretched too far, the model sometimes fabricated nonexistent fields.

 Lessons Learned:
- Prompt structure is crucial — clarity improves accuracy.
- Schema context must be tightly aligned with the question.
- Even well-trained models require human review to catch edge cases and hallucinations.
- Fun/fandom-based schemas can still be effective testbeds for prompt engineering and SQL logic.

Summary:
The experiment confirmed that transformer models can generate SQL accurately with the right
guidance. Prompt creativity is valuable, but must remain grounded in schema logic.
This lab demonstrated both the power and the limits of LLMs in code generation.
"""
